In [1]:
import torch
from pytorch_transformers import *
import numpy as np
# import pandas as pd
from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange as trange
from torch import nn
import random
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import run_squad as rs
# from utils_squad import (read_squad_examples, convert_examples_to_features,
#                          RawResult, write_predictions,
#                          RawResultExtended, write_predictions_extended)
import utils_squad as us
import nltk
from nltk import word_tokenize
from rouge import Rouge

In [2]:
import json
import logging
import math
import collections
from io import open

from pytorch_transformers.tokenization_bert import BasicTokenizer, whitespace_tokenize

# Required by XLNet evaluation method to compute optimal threshold (see write_predictions_extended() method)
from utils_squad_evaluate import find_all_best_thresh_v2, make_qid_to_has_ans, get_raw_scores

logger = logging.getLogger(__name__)

In [3]:
MODELS = [(BertForQuestionAnswering,       BertTokenizer,      'bert-large-uncased')]
# MODELS = [(BertForQuestionAnswering,       BertTokenizer,      'bert-large-uncased-whole-word-masking-finetuned-squad')]

In [6]:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    config = BertConfig.from_pretrained(pretrained_weights)


In [7]:
path = '/home/ubuntu/rl_testing/pytorch-transformers/examples/rl_state_dict.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
class RL_optimizer(BertForQuestionAnswering):
    def __init__(self, config):
        super(RL_optimizer, self).__init__(config)
        self.reward = 0
        self.bertqa = BertForQuestionAnswering(config)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, start_positions=None,
                end_positions=None, position_ids=None, head_mask=None):
        
        outputs = self.bertqa(self, input_ids, token_type_ids=None, attention_mask=None, start_positions=None,
                end_positions=None, position_ids=None, head_mask=None)

        return outputs

In [ ]:
rl = RL_optimizer(config)

In [8]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=0.9)



In [7]:
parallel_model = nn.DataParallel(model)

In [ ]:
# def calc_rewards():
#     """For now, we're using random rewards to check if the flow works fine"""
#     return random.choice([-0.5, 0.5])

In [9]:
class args_list:
    def __init__(self):
        self.local_rank = -1
        self.n_gpu = 1
        self.train_file = '/home/ubuntu/question_generation/data/train-v2.0.json'
        self.evaluate = 0
        self.predict_file = '/home/ubuntu/question_generation/data/dev-v2.0.json'
        self.eval_batch_size = 1
        self.model_type = 'bert'
        self.model_name_or_path = 'bert-base-uncased'
        self.output_dir = './outputs/'
        self.tokenizer_name = 'BertTokenizer'
        self.max_seq_length = 384
        self.version_2_with_negative = True
        self.doc_stride = 128
        self.max_query_length = 64
        self.device = torch.device('cuda')
        self.overwrite_cache = False
        self.null_score_diff_threshold = 0.0
        self.n_best_size = 20
        self.max_answer_length = 30
        self.verbose_logging = False
        self.do_lower_case = True
        

In [10]:
args = args_list()
args.device

device(type='cuda')

In [11]:
# torch.cuda.set_device(args.device)
model.to(args.device)
# rl.to(args.device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fals

In [12]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True
# torch.distributed.init_process_group(backend='nccl')

In [ ]:
## For Train
# dataset = rs.load_and_cache_examples(args, tokenizer)
dataset, examples, features = rs.load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=True, number_of_examples = 5)
train_sampler = SequentialSampler(dataset)
train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=8)

In [ ]:
## For Eval
dataset, examples, features = rs.load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)
eval_sampler = SequentialSampler(dataset) if args.local_rank == -1 else DistributedSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

In [13]:
def load_dataset():
    ## For Train
    # dataset = rs.load_and_cache_examples(args, tokenizer)
    dataset, examples, features = rs.load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=True, number_of_examples = 5)
    train_sampler = SequentialSampler(dataset) if args.local_rank == -1 else DistributedSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=2)
    return dataset, examples, features, train_sampler, train_dataloader

In [14]:
dataset, examples, features, train_sampler, train_dataloader = load_dataset()

In [15]:
def train_with_rewards(model, epochs = 2):
    
    train_iterator = trange(int(epochs), desc="Epoch", disable=-1 not in [-1, 0])
    model.zero_grad()
    
    for tr_iter in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=-1 not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            torch.cuda.empty_cache()
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
#             break
            outputs = model(**form_inputs(batch))
            loss = outputs[0]
            
            results, used_features = get_all_results(batch, outputs)
            used_examples = get_required_examples(used_features, examples)
            all_predictions = write_predictions(used_examples, used_features, results, args.n_best_size,
                        args.max_answer_length, args.do_lower_case, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file, args.verbose_logging,
                        args.version_2_with_negative, args.null_score_diff_threshold)
            
            rewards = calc_rewards(all_predictions, used_examples)
            loss = loss + rewards
            if step % 2000 == 0:
                print('reward = ', rewards)
            loss.backward()
            optimizer.step()
            model.zero_grad()
        print(f'The loss in step ', tr_iter, ' is ', loss)

In [21]:
train_with_rewards(model, epochs = 3)

reward =  2.0


/home/ubuntu/rl_testing/RL/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/rl_testing/RL/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/rl_testing/RL/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order

The loss in step  0  is  tensor(1.0073, device='cuda:0', grad_fn=<AddBackward0>)


reward =  2.0
The loss in step  1  is  tensor(1.0008, device='cuda:0', grad_fn=<AddBackward0>)


reward =  2.0
The loss in step  2  is  tensor(1.0004, device='cuda:0', grad_fn=<AddBackward0>)



In [16]:
train_iterator = trange(int(1), desc="Epoch", disable=-1 not in [-1, 0])
for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=-1 not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            model.train()
#             print(batch)
            batch = tuple(t.to(args.device) for t in batch)
#             print(batch)
            break

In [22]:
torch.save(model.state_dict(), 'rl_state_dict.pt')

In [18]:
def form_inputs(batch):
    inputs = {}
    inputs = {'input_ids': batch[0],
                      'attention_mask':  batch[1], 
                      'token_type_ids':  None if args.model_type == 'xlm' else batch[2],  
                      'start_positions': batch[3], 
                      'end_positions':   batch[4]}
    return inputs

In [19]:
outputs = model(**form_inputs(batch))

In [20]:
outputs

(tensor(0.8115, device='cuda:0', grad_fn=<DivBackward0>),
 tensor([[ -5.8113,  -9.5784, -10.2133, -10.1675, -10.2139, -10.2775, -10.6017,
          -10.6953, -10.4109,  -4.7854,  -8.3866,  -9.7981,  -7.0030,  -9.3320,
           -9.1561,  -9.5619,  -9.8994,  -9.6349,  -6.5670, -10.3656, -10.0256,
           -6.6386,  -9.7728, -10.7530, -10.6535,  -8.6792, -10.4406,  -9.9447,
          -10.0494, -10.2658,  -9.5349, -10.8343,  -8.2652,  -8.2829,  -5.7821,
           -8.3685,  -8.4558,  -5.2630,  -9.5228, -10.0244, -10.7324, -10.1083,
           -8.6699,  -9.9736,  -9.4496,  -9.7402,  -9.6530,  -9.9214, -10.3332,
          -10.0907,  -9.9857,  -8.7563, -10.6464,  -9.7717,  -9.3555,  -7.7645,
           -9.7250,  -7.8169,  -8.9943,  -9.7767,  -9.7294, -10.3602,  -8.8986,
           -7.7574,  -9.9791,  -8.6837,  -9.3785,  -9.0260, -10.0453,  -9.3273,
           -8.8221,  -7.6876,  -5.8835,  -7.3954,  -6.4988,   2.8644,   3.6163,
            5.8858,   3.4183,  -7.1160,  -7.2743,  -8.2778, -1

In [21]:
def get_all_results(batch, outputs):
    example_indices = batch[3]
    all_results = []
    used_features = []
    for i, example_index in enumerate(example_indices):
#                 print(example_index)
                eval_feature = features[example_index.item()]
                used_features.append(eval_feature)
                unique_id = int(eval_feature.unique_id)
                if args.model_type in ['xlnet', 'xlm']:
                    # XLNet uses a more complex post-processing procedure
                    result = us.RawResultExtended(unique_id            = unique_id,
                                               start_top_log_probs  = outputs[0][i].tolist(),
                                               start_top_index      = (outputs[1][i]).tolist(),
                                               end_top_log_probs    = (outputs[2][i]).tolist(),
                                               end_top_index        = (outputs[3][i]).tolist(),
                                               cls_logits           = (outputs[4][i]).tolist())
                else:
                    result = us.RawResult(unique_id    = unique_id,
                                       start_logits = outputs[1][i].tolist(),
                                       end_logits   = outputs[2][i].tolist())
#                     print(unique_id)
                all_results.append(result)
    return all_results, used_features

In [22]:
results, used_features = get_all_results(batch, outputs)

In [23]:
def get_required_examples(used_features, all_examples):
    used_examples = []
    for f in used_features:
        used_examples.append(all_examples[f.example_index])
    return used_examples

In [24]:
used_examples = get_required_examples(used_features, examples)

In [26]:
import os
prefix = ''
output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
if args.version_2_with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))
else:
        output_null_log_odds_file = None

In [25]:
all_predictions = write_predictions(used_examples, used_features, results, args.n_best_size,
                        args.max_answer_length, args.do_lower_case, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file, args.verbose_logging,
                        args.version_2_with_negative, args.null_score_diff_threshold)

NameError: name 'write_predictions' is not defined

In [27]:
def write_predictions(all_examples, all_features, all_results, n_best_size,
                      max_answer_length, do_lower_case, output_prediction_file,
                      output_nbest_file, output_null_log_odds_file, verbose_logging,
                      version_2_with_negative, null_score_diff_threshold):
    """Write final predictions to the json file and log-odds of null if needed."""
    logger.info("Writing predictions to: %s" % (output_prediction_file))
    logger.info("Writing nbest to: %s" % (output_nbest_file))

    example_index_to_features = collections.defaultdict(list)
    count_f = 0
    for feature in all_features:
        example_index_to_features[count_f].append(feature)
        count_f += 1

    unique_id_to_result = {}
    for result in all_results:
        unique_id_to_result[result.unique_id] = result

    _PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "PrelimPrediction",
        ["feature_index", "start_index", "end_index", "start_logit", "end_logit"])

    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    scores_diff_json = collections.OrderedDict()

    for (example_index, example) in enumerate(all_examples):
        features = example_index_to_features[example_index]

        prelim_predictions = []
        # keep track of the minimum score of null start+end of position 0
        score_null = 1000000  # large and positive
        min_null_feature_index = 0  # the paragraph slice with min null score
        null_start_logit = 0  # the start logit at the slice with min null score
        null_end_logit = 0  # the end logit at the slice with min null score
        for (feature_index, feature) in enumerate(features):
            
#             if feature.unique_id == 1000000075:
                
            result = unique_id_to_result[feature.unique_id]
            start_indexes = us._get_best_indexes(result.start_logits, n_best_size)
            end_indexes = us._get_best_indexes(result.end_logits, n_best_size)
            # if we could have irrelevant answers, get the min score of irrelevant
            if version_2_with_negative:
                feature_null_score = result.start_logits[0] + result.end_logits[0]
                if feature_null_score < score_null:
                    score_null = feature_null_score
                    min_null_feature_index = feature_index
                    null_start_logit = result.start_logits[0]
                    null_end_logit = result.end_logits[0]
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # We could hypothetically create invalid predictions, e.g., predict
                    # that the start of the span is in the question. We throw out all
                    # invalid predictions.
                    if start_index >= len(feature.tokens):
                        continue
                    if end_index >= len(feature.tokens):
                        continue
                    if start_index not in feature.token_to_orig_map:
                        continue
                    if end_index not in feature.token_to_orig_map:
                        continue
                    if not feature.token_is_max_context.get(start_index, False):
                        continue
                    if end_index < start_index:
                        continue
                    length = end_index - start_index + 1
                    if length > max_answer_length:
                        continue
                    prelim_predictions.append(
                        _PrelimPrediction(
                            feature_index=feature_index,
                            start_index=start_index,
                            end_index=end_index,
                            start_logit=result.start_logits[start_index],
                            end_logit=result.end_logits[end_index]))
        if version_2_with_negative:
            prelim_predictions.append(
                _PrelimPrediction(
                    feature_index=min_null_feature_index,
                    start_index=0,
                    end_index=0,
                    start_logit=null_start_logit,
                    end_logit=null_end_logit))
        prelim_predictions = sorted(
            prelim_predictions,
            key=lambda x: (x.start_logit + x.end_logit),
            reverse=True)

        _NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
            "NbestPrediction", ["text", "start_logit", "end_logit"])

        seen_predictions = {}
        nbest = []
        for pred in prelim_predictions:
            if len(nbest) >= n_best_size:
                break
            feature = features[pred.feature_index]
            if pred.start_index > 0:  # this is a non-null prediction
                tok_tokens = feature.tokens[pred.start_index:(pred.end_index + 1)]
                orig_doc_start = feature.token_to_orig_map[pred.start_index]
                orig_doc_end = feature.token_to_orig_map[pred.end_index]
                orig_tokens = example.doc_tokens[orig_doc_start:(orig_doc_end + 1)]
                tok_text = " ".join(tok_tokens)

                # De-tokenize WordPieces that have been split off.
                tok_text = tok_text.replace(" ##", "")
                tok_text = tok_text.replace("##", "")

                # Clean whitespace
                tok_text = tok_text.strip()
                tok_text = " ".join(tok_text.split())
                orig_text = " ".join(orig_tokens)

                final_text = us.get_final_text(tok_text, orig_text, do_lower_case, verbose_logging)
                if final_text in seen_predictions:
                    continue

                seen_predictions[final_text] = True
            else:
                final_text = ""
                seen_predictions[final_text] = True

            nbest.append(
                _NbestPrediction(
                    text=final_text,
                    start_logit=pred.start_logit,
                    end_logit=pred.end_logit))
        # if we didn't include the empty option in the n-best, include it
        if version_2_with_negative:
            if "" not in seen_predictions:
                nbest.append(
                    _NbestPrediction(
                        text="",
                        start_logit=null_start_logit,
                        end_logit=null_end_logit))
                
            # In very rare edge cases we could only have single null prediction.
            # So we just create a nonce prediction in this case to avoid failure.
            if len(nbest)==1:
                nbest.insert(0,
                    _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

        # In very rare edge cases we could have no valid predictions. So we
        # just create a nonce prediction in this case to avoid failure.
        if not nbest:
            nbest.append(
                _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

        assert len(nbest) >= 1

        total_scores = []
        best_non_null_entry = None
        for entry in nbest:
            total_scores.append(entry.start_logit + entry.end_logit)
            if not best_non_null_entry:
                if entry.text:
                    best_non_null_entry = entry

        probs = us._compute_softmax(total_scores)

        nbest_json = []
        for (i, entry) in enumerate(nbest):
            output = collections.OrderedDict()
            output["text"] = entry.text
            output["probability"] = probs[i]
            output["start_logit"] = entry.start_logit
            output["end_logit"] = entry.end_logit
            nbest_json.append(output)

        assert len(nbest_json) >= 1

        if not version_2_with_negative:
            all_predictions[example.qas_id] = nbest_json[0]["text"]
        else:
            # predict "" iff the null score - the score of best non-null > threshold
            score_diff = score_null - best_non_null_entry.start_logit - (
                best_non_null_entry.end_logit)
            scores_diff_json[example.qas_id] = score_diff
            if score_diff > null_score_diff_threshold:
                all_predictions[example.qas_id] = ""
            else:
                all_predictions[example.qas_id] = best_non_null_entry.text
        all_nbest_json[example.qas_id] = nbest_json

#     with open(output_prediction_file, "w") as writer:
#         writer.write(json.dumps(all_predictions, indent=4) + "\n")

#     with open(output_nbest_file, "w") as writer:
#         writer.write(json.dumps(all_nbest_json, indent=4) + "\n")

#     if version_2_with_negative:
#         with open(output_null_log_odds_file, "w") as writer:
#             writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [29]:
!ls

RL-test.ipynb	  rl_state_dict.pt	test_examples.py
RL_server.ipynb   run_bertology.py	tests_samples
__pycache__	  run_generation.py	utils_glue.py
lm_finetuning	  run_glue.py		utils_squad.py
outputs		  run_squad.py		utils_squad_evaluate.py
requirements.txt  single_model_scripts


In [ ]:
!mkdir outputs

In [ ]:
used_examples

In [ ]:
list(all_predictions.items())

In [ ]:
tokenizer.decode(batch[0][0].flatten().tolist())

In [18]:
def get_required_examples(used_features, all_examples):
    used_examples = []
    for f in used_features:
        used_examples.append(all_examples[f.example_index])
    return used_examples

In [20]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ['RANK'] = '0'
print(os.environ['CUDA_VISIBLE_DEVICES'])

0


In [ ]:
epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=-1 not in [-1, 0])
for step, batch in enumerate(epoch_iterator):
    batch = tuple(t.to(args.device) for t in batch)
    break

In [ ]:
batch

In [28]:
outputs = model(**form_inputs(batch))
loss = outputs[0]
            
results, used_features = get_all_results(batch, outputs)
used_examples = get_required_examples(used_features, examples)
all_predictions = write_predictions(used_examples, used_features, results, args.n_best_size,
                        args.max_answer_length, args.do_lower_case, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file, args.verbose_logging,
                        args.version_2_with_negative, args.null_score_diff_threshold)

In [29]:
list(all_predictions.items())

[('56d45abf2ccc5a1400d830e9', '&B'),
 ('56bf79c73aeaaa14008c966f', 'talent show circuit')]

In [ ]:
used_examples

In [31]:
def calc_rewards(all_predictions, used_examples):
    ## Convert all_predictions into list
    pred_list = list(all_predictions.items()) ## List of tuples
    ans_list = []
    bleu_loss = []
    rougel_f_scores = []
    rouge = Rouge()
    for id_x, ans in pred_list:
        ans_list.append(word_tokenize(ans)) ## List of all the answers. shape - (batch_size,)
    
    actual_answers = []
    
    for ex in used_examples:
        st = ex.start_position
        end = ex.end_position
        actual_ans = ex.doc_tokens[st:end]
        actual_answers.append(actual_ans) ## List of actual answers from train set
   
    for i in range(len(ans_list)):
        bleu_loss.append(1 - nltk.translate.bleu_score.sentence_bleu([actual_answers[i]], ans_list[i]))
        try:
            scores = rouge.get_scores(" ".join(ans_list[i]), " ".join(actual_answers[i]))
            rougel_f_scores.append(1 - scores[0]['rouge-l']['f'])
        except:
            rougel_f_scores.append(0.0)
#         print(scores)
        
    bleu_avg = sum(bleu_loss)/len(bleu_loss)
    rouge_avg = sum(rougel_f_scores)/len(rougel_f_scores)
    
    total_loss = bleu_avg + rouge_avg
        
    return total_loss
        

In [32]:
bls = calc_rewards(all_predictions, used_examples)
print(bls)

2.0


In [ ]:
start_logits = outputs[1][0]
start_logits_sq = start_logits.squeeze(-1)

In [ ]:
batch[3]

In [ ]:
tokenizer.decode([75])

In [ ]:
start_logits_sq

In [ ]:
tok = used_examples[0].doc_tokens
" ".join(tok)

In [ ]:
all_predictions

In [ ]:
used_examples

In [ ]:
used_examples[1].doc_tokens[used_examples[1].start_position : used_examples[1].end_position]

In [ ]:
torch.cuda.device_count()